In [ ]:
!pip install --quiet img2vec_pytorch    # görüntüleri vektör temsillerine dönüştürmek için, sebebi: görüntüdeki her bir pikselin RGB değerlerini önce binary haline convert edip sonra her bir pikselin binary değerlerinin sayısal karşılığını tek bir vektorde tutuyor
!pip install --quiet arrow               #tarih ve zaman işlemleri için
!pip install umap-learn                  # vektör temsillerini daha düşük boyutlu bir uzaya projeksiyonlar. Daha sonra, bu düşük boyutlu uzayda görselleştirmek için iki boyutlu koordinatlara indirgenmiş olan veriyi içeren bir DataFrame oluşturur.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


resnet18 ile feature extraction

In [ ]:
from img2vec_pytorch import Img2Vec
from PIL import Image                  #PIL, resim dosyalarını açma, kaydetme, dönüştürme ve işleme gibi bir dizi işlevi içerir.
from arrow import now
from glob import glob             #Örneğin, "*.txt" deseni, bir dizindeki tüm .txt uzantılı dosyaları eşleştirir.
import pandas as pd                     # Veri analizi ve manipülasyonu için
from os.path import basename             #bir dosya yolundan dosya adını almak için kullanılır.

img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)     #Img2Vec sınıfından bir nesne oluşturuyoruz. Bu nesne, görüntülerin vektör temsillerini elde etmek için kullanılacak. ResNet de CNN ailesine ait bir modeldir.
                                                                                                     #ResNet-18 gibi modellerde, belirli katmanların çıkışları belirli boyutlarda özellik vektörleri üretir.  modelin o katmanından çıkarılan özellik vektörünün 512 boyutlu olacağını gösterir.
GLOB_Benign = '/content/drive/MyDrive/melonamacancer/train/Benign/*'
GLOB_Malignant = '/content/drive/MyDrive/melonamacancer/train/Malignant/*'

def get_from_glob(arg: str, tag: str) -> list:                  #arg, işlenecek dosyaların yolu ve tag, her resmin hangi kategoriye ait olduğu
    result = []
    for input_file in glob(pathname=arg):                        #glob ile belirtilen dosya yolundaki (benign veya melignant klasörleri) her bir dosya işleme girer sırasıyla
        name = basename(input_file)
        try:
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
        except RuntimeError:
            # we only have a few failures so we're just going to discard them
            pass
    return result

time_start = now()
Benign = get_from_glob(arg=GLOB_Benign, tag='benign')
print('done encoding the benign images in {}'.format(now() - time_start))
Malignant = get_from_glob(arg=GLOB_Malignant, tag='malignant')
df = pd.DataFrame(data=Benign + Malignant)
print('done in {}'.format(now() - time_start))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 91.3MB/s]


done encoding the benign images in 0:09:48.518733
done in 0:18:14.519836


In [ ]:
df

,tag,name,value
0,benign,5765.jpg,"[1.8563226, 0.8769158, 0.0, 2.4691143, 0.04565..."
1,benign,5764.jpg,"[0.8542331, 0.010713275, 0.0, 1.8745655, 0.0, ..."
2,benign,5768.jpg,"[1.0994872, 0.13306837, 0.05266045, 1.2007028,..."
3,benign,5766.jpg,"[0.4403347, 0.0049412167, 0.22826391, 0.623122..."
4,benign,5767.jpg,"[1.6859819, 0.34678423, 0.065606825, 0.883151,..."
...,...,...,...
11925,malignant,1569.jpg,"[0.7718343, 0.06514406, 0.069201864, 1.6364101..."
11926,malignant,1573.jpg,"[2.545112, 0.66796726, 0.27408704, 1.0507064, ..."
11927,malignant,1584.jpg,"[0.9670818, 0.28629, 0.3422997, 1.0374728, 0.0..."
11928,malignant,1590.jpg,"[1.7722458, 0.42369357, 0.35403952, 0.8382542,..."


In [ ]:
from umap import UMAP
umap = UMAP(random_state=2024, verbose=True, n_jobs=1,)
model_df = pd.DataFrame(data=umap.fit_transform(X=df['value'].apply(pd.Series)), columns=['x', 'y'])            #value değerlerinin 2D e indirgenmiş hallerini sadece grafikte datapoint lerin dağılımını görmede kuallanıcaz
plot_df = pd.concat(objs=[df, model_df], axis=1)                                                                # modeli eğitirken 2D değerlerini değil normal herbir image için elde edilmiş vector değerlerini kullanacağız
print('done with UMAP in {}'.format(now() - time_start))

UMAP(n_jobs=1, random_state=2024, verbose=True)
Mon May 20 11:17:58 2024 Construct fuzzy simplicial set
Mon May 20 11:17:58 2024 Finding Nearest Neighbors
Mon May 20 11:17:58 2024 Building RP forest with 10 trees
Mon May 20 11:18:03 2024 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	Stopping threshold met -- exiting after 5 iterations
Mon May 20 11:18:19 2024 Finished Nearest Neighbor Search
Mon May 20 11:18:22 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Mon May 20 11:18:33 2024 Finished embedding
done with UMAP in 0:19:12.556190


In [ ]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='x', y='y', color='tag', hover_name='name', height=900)

In [ ]:
img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)     #Img2Vec sınıfından bir nesne oluşturuyoruz. Bu nesne, görüntülerin vektör temsillerini elde etmek için kullanılacak. ResNet de CNN ailesine ait bir modeldir.

GLOB_Benign_test = '/content/drive/MyDrive/melonamacancer/test/Benign/*'
GLOB_Malignant_test = '/content/drive/MyDrive/melonamacancer/test/Malignant/*'

def get_from_glob(arg: str, tag: str) -> list:                  #arg, işlenecek dosyaların yolu ve tag, her resmin hangi kategoriye ait olduğu
    result = []
    for input_file in glob(pathname=arg):                        #glob ile belirtilen dosya yolundaki (ai veya real klasörleri) her bir dosya işleme girer sırasıyla
        name = basename(input_file)
        try:
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512,)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
        except RuntimeError:
            # we only have a few failures so we're just going to discard them
            pass
    return result

time_start = now()
Benign_test = get_from_glob(arg=GLOB_Benign_test, tag='benign')
print('done encoding the images in {}'.format(now() - time_start))
Malignant_test = get_from_glob(arg=GLOB_Malignant_test, tag='malignant')
df_test = pd.DataFrame(data=Benign_test + Malignant_test)
print('done in {}'.format(now() - time_start))

X_test=df_test['value'].apply(pd.Series)
y_test=df_test['tag']

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.



done encoding the images in 0:01:34.860749
done in 0:03:02.500623


In [ ]:
#test setini görselleştirme
umap = UMAP(random_state=2024, verbose=True, n_jobs=1,)
model_df_test = pd.DataFrame(data=umap.fit_transform(X=df_test['value'].apply(pd.Series)), columns=['x', 'y'])            #value değerlerinin 2D e indirgenmiş hallerini sadece grafikte datapoint lerin dağılımını görmede kuallanıcaz
plot_df_test = pd.concat(objs=[df_test, model_df_test], axis=1)                                                                # modeli eğitirken 2D değerlerini değil normal herbir image için elde edilmiş vector değerlerini kullanacağız
print('done with UMAP in {}'.format(now() - time_start))

UMAP(n_jobs=1, random_state=2024, verbose=True)
Mon May 20 11:21:38 2024 Construct fuzzy simplicial set
Mon May 20 11:21:40 2024 Finding Nearest Neighbors
Mon May 20 11:21:44 2024 Finished Nearest Neighbor Search
Mon May 20 11:21:44 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Mon May 20 11:21:48 2024 Finished embedding
done with UMAP in 0:03:13.352309


In [ ]:
scatter(data_frame=plot_df_test, x='x', y='y', color='tag', hover_name='name', height=900)

In [ ]:
df_test

,tag,name,value
0,benign,6299.jpg,"[0.39094874, 0.8595761, 0.16068968, 1.1681559,..."
1,benign,6300.jpg,"[0.5516767, 0.040642023, 0.07764553, 2.0382063..."
2,benign,6301.jpg,"[1.6560893, 0.59010327, 0.051712174, 1.2118062..."
3,benign,6311.jpg,"[2.4047742, 0.4185637, 0.014620412, 0.726816, ..."
4,benign,6312.jpg,"[0.20154724, 1.5773513, 0.0, 1.6303614, 0.4809..."
...,...,...,...
2015,malignant,5612.jpg,"[0.64098334, 0.38275668, 0.08618794, 0.4469243..."
2016,malignant,5613.jpg,"[3.303083, 0.028153792, 0.1810992, 2.0821433, ..."
2017,malignant,5608.jpg,"[1.6141052, 0.7547754, 0.08927307, 0.72970676,..."
2018,malignant,5621 (1).jpg,"[1.9083126, 0.38536608, 0.9399328, 0.5957865, ..."


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train=df['value'].apply(pd.Series)
y_train=df['tag']

model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)


print('accuracy: {:5.2f} pct'.format(100 * accuracy_score(y_test, model.predict(X_test))))

accuracy: 88.47 pct


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test, y_pred = model.predict(X_test)))

              precision    recall  f1-score   support

      benign       0.85      0.93      0.89      1000
   malignant       0.92      0.84      0.88      1020

    accuracy                           0.88      2020
   macro avg       0.89      0.89      0.88      2020
weighted avg       0.89      0.88      0.88      2020



In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1.856323,0.876916,0.000000,2.469114,0.045651,1.881800,0.152968,0.000000,1.140911,0.707484,...,0.094814,0.005453,0.041351,3.867677,1.360564,0.678261,0.000000,1.990491,0.000000,0.008800
1,0.854233,0.010713,0.000000,1.874565,0.000000,0.845564,0.522560,0.076347,2.352181,0.079837,...,0.000000,0.000000,0.000309,4.424758,1.509570,0.419274,0.325524,1.362803,0.000000,0.049178
2,1.099487,0.133068,0.052660,1.200703,0.146729,0.094476,0.022364,0.049750,2.010974,0.697842,...,0.196732,0.462259,1.461802,0.470899,1.019254,0.850166,0.093610,0.450334,0.012976,0.068848
3,0.440335,0.004941,0.228264,0.623123,0.054811,0.023991,0.054350,0.000000,2.112791,0.397828,...,0.188505,2.485912,1.164745,0.838622,0.220581,0.753696,0.205527,0.536324,0.000000,0.020699
4,1.685982,0.346784,0.065607,0.883151,0.175851,0.337840,0.872156,0.000000,1.018934,0.312650,...,0.004156,0.581071,0.102258,1.718815,1.865599,1.112972,0.074934,0.488672,0.291185,0.336140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11925,0.771834,0.065144,0.069202,1.636410,0.847109,1.326262,0.020724,0.054497,0.212178,0.247941,...,0.107603,0.684265,0.373855,1.112921,0.164951,2.310757,0.018894,0.234253,0.033843,0.088400
11926,2.545112,0.667967,0.274087,1.050706,0.028738,0.411645,0.090467,0.054598,0.609388,1.755733,...,0.027854,0.496310,0.059449,2.536213,1.531932,1.071998,0.011391,0.995675,1.115800,0.174259
11927,0.967082,0.286290,0.342300,1.037473,0.068702,0.386223,0.857385,0.112654,0.030857,2.527744,...,0.090933,1.840933,0.766624,0.722599,1.740136,0.438958,0.212156,0.036841,0.016914,0.217950
11928,1.772246,0.423694,0.354040,0.838254,0.009172,0.196930,0.268649,0.448420,0.205557,0.345797,...,0.000000,0.390742,0.110985,0.192989,0.273942,1.347379,0.027874,0.996099,1.074637,1.406711


cnn 512 lik fature vector ile besleme

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Sınıf etiketlerini sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Yeni bir sınıflandırma modeli oluşturma
model = Sequential([
    Dense(512, activation='relu', input_shape=(512,)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Erken durdurma çağrısı
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Modeli eğitme
history = model.fit(X_train, y_train_encoded, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Modelin performansını değerlendirme
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test accuracy:', test_acc)


# Tahminleri ikili sınıf etiketlerine dönüştürme
y_pred_binary = np.where(model.predict(X_test) > 0.5, 1, 0)
# Sınıflandırma raporu oluşturma
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test_encoded, y_pred=y_pred_binary))

Epoch 1/20
336/336 [==============================] - 5s 5ms/step - loss: 0.3502 - accuracy: 0.8555 - val_loss: 0.5496 - val_accuracy: 0.7603
Epoch 2/20
336/336 [==============================] - 1s 4ms/step - loss: 0.2881 - accuracy: 0.8813 - val_loss: 0.7457 - val_accuracy: 0.6303
Epoch 3/20
336/336 [==============================] - 2s 5ms/step - loss: 0.2733 - accuracy: 0.8874 - val_loss: 0.4857 - val_accuracy: 0.7963
Epoch 4/20
336/336 [==============================] - 2s 6ms/step - loss: 0.2578 - accuracy: 0.8898 - val_loss: 0.3926 - val_accuracy: 0.8039
Epoch 5/20
336/336 [==============================] - 2s 5ms/step - loss: 0.2552 - accuracy: 0.8942 - val_loss: 0.6505 - val_accuracy: 0.6857
Epoch 6/20
336/336 [==============================] - 1s 4ms/step - loss: 0.2398 - accuracy: 0.8983 - val_loss: 0.4693 - val_accuracy: 0.8089
Epoch 7/20
64/64 [==============================] - 0s 3ms/step - loss: 0.3194 - accuracy: 0.8629
Test accuracy: 0.8628712892532349
64/64 [=========

ANN İLE MODEL EĞİTİMİ

In [ ]:
from PIL import Image

example_train_image = '/content/drive/MyDrive/melonamacancer/train/Benign/1.jpg'
image = Image.open(example_train_image)
# Görüntünün boyutunu kontrol etme
print("Görüntünün boyutu:", np.array(image).shape)

Görüntünün boyutu: (224, 224, 3)


raw data çıkarımı

In [ ]:
import os
from PIL import Image
import numpy as np

def load_images_with_labels(image_dir, label):
    X_images = []
    y_labels = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Görüntü dosyalarını filtreleme
            image_path = os.path.join(image_dir , filename)
            image = Image.open(image_path)
            # Görüntü boyutlarını ve kanal sayısını 28x28x3 olarak ayarlama
            image = image.resize((28, 28))  #28x28 boyutuna yeniden boyutlandırma   #BU SATIRI SİLİNCE CNN DE 224,224,3 COLAB I ÇÖKTÜRÜYOR :(
            # Görüntüyü numpy dizisine dönüştür ve listeye ekle
            X_images.append(np.array(image))
            y_labels.append(label)  # Etiketi ekle

    return X_images, y_labels

# Benign görüntüleri yükle
X_train_benign, y_train_benign = load_images_with_labels('/content/drive/MyDrive/melonamacancer/train/Benign', 'Benign')

# Malignant görüntüleri yükle
X_train_malignant, y_train_malignant = load_images_with_labels('/content/drive/MyDrive/melonamacancer/train/Malignant', 'Malignant')

In [ ]:
# Tüm görüntüleri birleştir
X_train_images = X_train_benign + X_train_malignant
y_train_labels = y_train_benign + y_train_malignant

In [ ]:
def load_images_with_labels(image_dir, label):
    X_images = []
    y_labels = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Görüntü dosyalarını filtrele
            image_path = os.path.join(image_dir , filename)
            image = Image.open(image_path)
            # Görüntü boyutlarını ve kanal sayısını 28x28x3 olarak ayarla
            image = image.resize((28, 28))  # Örnek olarak 28x28 boyutuna yeniden boyutlandırma
            # Görüntüyü numpy dizisine dönüştür ve listeye ekle
            X_images.append(np.array(image))
            y_labels.append(label)  # Etiketi ekle

    return X_images, y_labels

# Benign görüntüleri yükleme
X_test_benign, y_test_benign = load_images_with_labels('/content/drive/MyDrive/melonamacancer/test/Benign', 'Benign')

# Malignant görüntüleri yükleme
X_test_malignant, y_test_malignant = load_images_with_labels('/content/drive/MyDrive/melonamacancer/test/Malignant', 'Malignant')

In [ ]:
# Tüm görüntüleri birleştirme
X_test_images = X_test_benign + X_test_malignant
y_test_labels = y_test_benign + y_test_malignant

In [ ]:
y_test_labels

['Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',
 'Benign',

In [ ]:
X_train_images

[array([[[202, 130, 143],
         [209, 136, 153],
         [217, 144, 163],
         ...,
         [217, 157, 172],
         [206, 150, 162],
         [184, 129, 136]],
 
        [[200, 129, 138],
         [205, 133, 146],
         [213, 140, 158],
         ...,
         [214, 154, 170],
         [208, 152, 164],
         [190, 135, 143]],
 
        [[202, 127, 136],
         [203, 131, 140],
         [207, 135, 150],
         ...,
         [215, 154, 170],
         [210, 152, 165],
         [195, 142, 149]],
 
        ...,
 
        [[194, 138, 139],
         [198, 143, 144],
         [201, 145, 146],
         ...,
         [210, 156, 156],
         [196, 149, 149],
         [175, 132, 131]],
 
        [[192, 136, 137],
         [199, 143, 144],
         [202, 146, 147],
         ...,
         [207, 153, 156],
         [192, 145, 145],
         [167, 124, 123]],
 
        [[189, 133, 134],
         [196, 140, 141],
         [199, 143, 144],
         ...,
         [203, 153, 154],
  

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# X_train_images ve X_test_images listelerini numpy dizilerine dönüştürün
X_train_images = np.array(X_train_images)
X_test_images = np.array(X_test_images)

# Verileri yeniden şekillendirme ve normalleştirme
X_train_images = X_train_images.reshape(11930, 28, 28, 3).astype('float32') / 255.0
X_test_images = X_test_images.reshape(2020, 28, 28, 3).astype('float32') / 255.0

# Etiketleri sayısal değerlere dönüşümü
le = LabelEncoder()
y_train_labels = le.fit_transform(y_train_labels)
y_test_labels = le.transform(y_test_labels)

# Etiketleri one-hot encoding ile kodlama
y_train_labels = to_categorical(y_train_labels, num_classes=2)
y_test_labels = to_categorical(y_test_labels, num_classes=2)

cnn i raw data ile besleme

In [ ]:
# CNN modelini oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train_images, y_train_labels, epochs=5, batch_size=5, validation_split=0.1)

# Modelin performansı
test_loss, test_acc = model.evaluate(X_test_images, y_test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5
2148/2148 [==============================] - 13s 5ms/step - loss: 0.4033 - accuracy: 0.8138 - val_loss: 0.8334 - val_accuracy: 0.5298
Epoch 2/5
2148/2148 [==============================] - 11s 5ms/step - loss: 0.3324 - accuracy: 0.8541 - val_loss: 0.7054 - val_accuracy: 0.6220
Epoch 3/5
2148/2148 [==============================] - 10s 5ms/step - loss: 0.3210 - accuracy: 0.8592 - val_loss: 1.0341 - val_accuracy: 0.4627
Epoch 4/5
2148/2148 [==============================] - 9s 4ms/step - loss: 0.3071 - accuracy: 0.8596 - val_loss: 0.8408 - val_accuracy: 0.5717
Epoch 5/5
64/64 [==============================] - 0s 5ms/step - loss: 0.3382 - accuracy: 0.8733
Test accuracy: 0.8732673525810242


ANN'i SIFT ile besleme